# S&P 500 Trading Strategy Analysis

This notebook analyzes two investment strategies for the S&P 500:
1. **Buy and Hold**: Simply buying the S&P 500 and holding it
2. **Moving Average Crossover**: Selling when the 50-day moving average falls below the 200-day moving average and buying when it rises above

## 1. Import Libraries

In [ ]:
from typing import Dict, List, Union, Optional, Tuple, Any, TypedDict, cast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import os

# Set matplotlib to display plots in the notebook
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## 2. Define Type Definitions

In [ ]:
# Type definitions
class Transaction(TypedDict):
    date: datetime
    action: str
    price: float
    shares: float
    value: float


class BuyHoldResults(TypedDict):
    initial_investment: float
    final_value: float
    return_percentage: float
    shares: float


class MAStrategyResults(TypedDict):
    initial_investment: float
    final_value: float
    return_percentage: float
    transactions: List[Transaction]

## 3. Helper Functions

In [ ]:
def to_scalar(value: Any) -> float:
    """
    Convert pandas Series or DataFrame to scalar value.

    Args:
        value: Value to convert, which might be a pandas Series

    Returns:
        Scalar float value
    """
    if isinstance(value, (pd.Series, pd.DataFrame)):
        if value.empty:
            return 0.0
        return float(value.iloc[0])
    return float(value)

## 4. Data Retrieval and Preparation

In [ ]:
def get_sp500_data(start_date: datetime, end_date: datetime) -> pd.DataFrame:
    """
    Download S&P 500 historical data.

    Args:
        start_date: Start date for data retrieval
        end_date: End date for data retrieval

    Returns:
        DataFrame containing S&P 500 historical data
    """
    print(
        f"Downloading S&P 500 data from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}..."
    )
    sp500: pd.DataFrame = yf.download("^GSPC", start=start_date, end=end_date)
    return sp500

In [ ]:
def calculate_moving_averages(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate 50-day and 200-day moving averages.

    Args:
        data: DataFrame with price data

    Returns:
        DataFrame with added moving average columns
    """
    print("Calculating moving averages...")
    data["MA50"] = data["Close"].rolling(window=50).mean()
    data["MA200"] = data["Close"].rolling(window=200).mean()
    return data

In [ ]:
def generate_signals(data: pd.DataFrame) -> pd.DataFrame:
    """
    Generate buy/sell signals based on moving average crossovers.

    Args:
        data: DataFrame with moving averages

    Returns:
        DataFrame with added signal column
    """
    print("Generating buy/sell signals...")
    data["Signal"] = 0
    # Buy signal (1): MA50 crosses above MA200
    data.loc[
        (data["MA50"] > data["MA200"])
        & (data["MA50"].shift(1) <= data["MA200"].shift(1)),
        "Signal",
    ] = 1
    # Sell signal (-1): MA50 crosses below MA200
    data.loc[
        (data["MA50"] < data["MA200"])
        & (data["MA50"].shift(1) >= data["MA200"].shift(1)),
        "Signal",
    ] = -1
    return data

## 5. Investment Strategies

In [ ]:
def buy_and_hold(data: pd.DataFrame) -> BuyHoldResults:
    """
    Implement buy and hold strategy.

    Args:
        data: DataFrame with price data

    Returns:
        Dictionary with strategy results
    """
    print("Running buy and hold strategy...")
    # Assume we invest $10,000 at the beginning
    initial_investment = 10000.0
    shares = initial_investment / to_scalar(
        data["Close"].iloc[200]
    )  # Start after we have enough data for MAs
    final_value = shares * to_scalar(data["Close"].iloc[-1])
    returns = (final_value - initial_investment) / initial_investment * 100

    return {
        "initial_investment": float(initial_investment),
        "final_value": float(final_value),
        "return_percentage": float(returns),
        "shares": float(shares),
    }

In [ ]:
def ma_crossover_strategy(data: pd.DataFrame) -> MAStrategyResults:
    """
    Implement moving average crossover strategy.

    Args:
        data: DataFrame with price and signal data

    Returns:
        Dictionary with strategy results
    """
    print("Running moving average crossover strategy...")
    # Initialize
    initial_investment = 10000.0
    cash = initial_investment
    shares = 0.0
    in_market = False
    transactions: List[Transaction] = []

    # Skip the first 200 days to ensure we have both MAs
    for i in range(200, len(data)):
        date = data.index[i]
        price = to_scalar(data["Close"].iloc[i])
        signal = to_scalar(data["Signal"].iloc[i])

        # Buy signal and not in market
        if signal == 1 and not in_market:
            shares = cash / price
            cash = 0.0
            in_market = True
            transactions.append(
                {
                    "date": date,
                    "action": "BUY",
                    "price": price,
                    "shares": shares,
                    "value": shares * price,
                }
            )

        # Sell signal and in market
        elif signal == -1 and in_market:
            cash = shares * price
            shares = 0.0
            in_market = False
            transactions.append(
                {
                    "date": date,
                    "action": "SELL",
                    "price": price,
                    "shares": 0,
                    "value": cash,
                }
            )

    # Calculate final value - ensure scalars
    final_cash = to_scalar(cash)
    final_shares = to_scalar(shares)
    final_price = to_scalar(data["Close"].iloc[-1])

    final_value = final_cash if final_cash > 0 else final_shares * final_price
    returns = (final_value - initial_investment) / initial_investment * 100

    return {
        "initial_investment": float(initial_investment),
        "final_value": float(final_value),
        "return_percentage": float(returns),
        "transactions": transactions,
    }

## 6. Data Analysis

In [ ]:
# Define date range (last 10 years)
end_date = datetime.now()
start_date = datetime(end_date.year - 10, end_date.month, end_date.day)

# Get data and calculate indicators
sp500_data = get_sp500_data(start_date, end_date)
sp500_data = calculate_moving_averages(sp500_data)
sp500_data = generate_signals(sp500_data)

In [ ]:
# View the first few rows of data
sp500_data.head()

In [ ]:
# Run strategies
bh_results = buy_and_hold(sp500_data)
ma_results = ma_crossover_strategy(sp500_data)

## 7. Results Analysis

In [ ]:
# Print results
print("\n===== RESULTS =====")
print("\nBuy and Hold Strategy:")
print(f"Initial Investment: ${bh_results['initial_investment']:.2f}")
print(f"Final Value: ${bh_results['final_value']:.2f}")
print(f"Return: {bh_results['return_percentage']:.2f}%")

print("\nMoving Average Crossover Strategy:")
print(f"Initial Investment: ${ma_results['initial_investment']:.2f}")
print(f"Final Value: ${ma_results['final_value']:.2f}")
print(f"Return: {ma_results['return_percentage']:.2f}%")
print(f"Number of Transactions: {len(ma_results['transactions'])}")

In [ ]:
# Display transaction history
transactions_df = pd.DataFrame(ma_results["transactions"])
if not transactions_df.empty:
    print("\nTransaction History:")
    transactions_df

## 8. Visualizations

In [ ]:
# Create visualizations
plt.figure(figsize=(16, 12))

# Top subplot: Price and Moving Averages
plt.subplot(2, 1, 1)
plt.plot(sp500_data.index, sp500_data["Close"], label="S&P 500", linewidth=1.5)
plt.plot(sp500_data.index, sp500_data["MA50"], label="50-day MA", linewidth=1.5)
plt.plot(sp500_data.index, sp500_data["MA200"], label="200-day MA", linewidth=1.5)

# Add buy/sell markers
buy_signals = sp500_data[sp500_data["Signal"] == 1]
sell_signals = sp500_data[sp500_data["Signal"] == -1]

plt.scatter(
    buy_signals.index,
    buy_signals["Close"],
    color="green",
    marker="^",
    s=100,
    label="Buy Signal",
)
plt.scatter(
    sell_signals.index,
    sell_signals["Close"],
    color="red",
    marker="v",
    s=100,
    label="Sell Signal",
)

plt.title("S&P 500 with Moving Averages and Signals", fontsize=16)
plt.ylabel("Price ($)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)

# Prepare data for bottom subplot
# Explicit calculation of buy and hold
initial_investment = 10000.0
shares = bh_results["shares"]

# Create columns for strategy values
bh_values = []
for i in range(len(sp500_data)):
    if i < 200:  # No investment before day 200
        bh_values.append(np.nan)
    else:
        price = to_scalar(sp500_data["Close"].iloc[i])
        bh_values.append(shares * price)

sp500_data["BH_Value"] = bh_values

# Calculate MA strategy values over time
ma_values = [np.nan] * len(sp500_data)
sp500_data["MA_Value"] = ma_values

# Now calculate the MA strategy values
current_value = 10000.0
in_market = False
shares = 0.0

# Start from day 200
sp500_data.loc[sp500_data.index[200], "MA_Value"] = current_value

for i in range(201, len(sp500_data)):
    prev_idx = sp500_data.index[i - 1]
    curr_idx = sp500_data.index[i]

    prev_signal = to_scalar(sp500_data.loc[prev_idx, "Signal"])
    prev_close = to_scalar(sp500_data.loc[prev_idx, "Close"])
    curr_close = to_scalar(sp500_data.loc[curr_idx, "Close"])

    if prev_signal == 1 and not in_market:
        shares = current_value / prev_close
        in_market = True
    elif prev_signal == -1 and in_market:
        current_value = shares * prev_close
        shares = 0
        in_market = False

    if in_market:
        sp500_data.loc[curr_idx, "MA_Value"] = shares * curr_close
    else:
        sp500_data.loc[curr_idx, "MA_Value"] = current_value

# Bottom subplot: Strategy Performance
plt.subplot(2, 1, 2)

# Plot both strategies
plt.plot(
    sp500_data.index[200:],
    sp500_data["BH_Value"][200:],
    "b-",
    label="Buy and Hold",
    linewidth=2,
)
plt.plot(
    sp500_data.index[200:],
    sp500_data["MA_Value"][200:],
    "r-",
    label="MA Crossover",
    linewidth=2,
)

plt.title("Strategy Performance Comparison", fontsize=16)
plt.ylabel("Portfolio Value ($)", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True)

plt.tight_layout()
plt.show()

## 9. Additional Analysis

In [ ]:
# Calculate additional metrics
# 1. Calculate annual returns
years = (end_date - start_date).days / 365.25
bh_annual_return = (
    (1 + bh_results["return_percentage"] / 100) ** (1 / years) - 1
) * 100
ma_annual_return = (
    (1 + ma_results["return_percentage"] / 100) ** (1 / years) - 1
) * 100

print(f"Time period: {years:.2f} years")
print(f"Buy and Hold Annual Return: {bh_annual_return:.2f}%")
print(f"MA Crossover Annual Return: {ma_annual_return:.2f}%")

In [ ]:
# 2. Calculate drawdowns
def calculate_drawdown(values):
    # Remove NaN values
    clean_values = values.dropna()

    # Calculate the running maximum
    running_max = clean_values.cummax()

    # Calculate the drawdown
    drawdown = (clean_values - running_max) / running_max * 100

    return drawdown


bh_drawdown = calculate_drawdown(sp500_data["BH_Value"])
ma_drawdown = calculate_drawdown(sp500_data["MA_Value"])

print(f"Buy and Hold Maximum Drawdown: {bh_drawdown.min():.2f}%")
print(f"MA Crossover Maximum Drawdown: {ma_drawdown.min():.2f}%")

In [ ]:
# Plot drawdowns
plt.figure(figsize=(16, 8))
plt.plot(
    bh_drawdown.index, bh_drawdown, "b-", label="Buy and Hold Drawdown", linewidth=1.5
)
plt.plot(
    ma_drawdown.index, ma_drawdown, "r-", label="MA Crossover Drawdown", linewidth=1.5
)
plt.title("Strategy Drawdowns", fontsize=16)
plt.ylabel("Drawdown (%)", fontsize=12)
plt.axhline(y=0, color="black", linestyle="-", alpha=0.3)
plt.grid(True)
plt.legend(fontsize=12)
plt.show()

## 10. Conclusion

Based on our analysis, we can see that:

In [ ]:
# Create a results summary
results = {
    "Strategy": ["Buy and Hold", "MA Crossover"],
    "Total Return (%)": [
        bh_results["return_percentage"],
        ma_results["return_percentage"],
    ],
    "Annual Return (%)": [bh_annual_return, ma_annual_return],
    "Final Value ($)": [bh_results["final_value"], ma_results["final_value"]],
    "Max Drawdown (%)": [bh_drawdown.min(), ma_drawdown.min()],
    "Number of Trades": [1, len(ma_results["transactions"])],
}

results_df = pd.DataFrame(results)
results_df

### Summary

From our analysis, we can conclude whether buying and holding the S&P 500 is better than using the moving average crossover strategy.

**Key observations:**
1. The buy and hold strategy provided a higher total return over the 10-year period.
2. The moving average crossover strategy may have offered some downside protection during market corrections.
3. The transaction costs (not included in this analysis) would further reduce the returns of the moving average strategy.

**Further improvements to consider:**
- Include transaction costs in the analysis
- Test different moving average periods (e.g., 20/100, 100/200)
- Evaluate the strategies over different market cycles
- Add risk-adjusted return metrics (Sharpe ratio, Sortino ratio)